# Frequency

Creating one big frequency table of Hungarian words. 
Sources: HNC (http://corpus.nytud.hu/mnsz/index_eng.html) and HWC (http://mokk.bme.hu/resources/webcorpus/)

In [ ]:
import pandas as pd

colnames=['word','freq','1','2','3']
df1 = pd.read_csv("web2.2-freq-sorted.txt", sep='\t',header=None, encoding='ISO-8859-2',names=colnames)
df1 = df1[['word','freq']]
df1 = df1.groupby(['word']).sum()
df1 = df1.sort_values('freq', ascending=False).reset_index()

colnames = ['word','1','2','3','4','5','6','freq']
df2 = pd.read_csv("hnc-1.3-wordfreq.txt", sep='\t',header=None,encoding="latin-1",names=colnames)
df2 = df2[['word','freq']]
df2 = df2.groupby(['word']).sum()
df2 = df2.sort_values('freq', ascending=False).reset_index()

# merge the two dataframes based on 'word'
merged_df = pd.merge(df1, df2, on='word', how='outer')

# group by 'word' and sum 'freq'
grouped_df = merged_df.groupby('word')['freq_x', 'freq_y'].sum().reset_index()

# set 'freq' to the sum of 'freq_x' and 'freq_y'
grouped_df['freq'] = grouped_df['freq_x'] + grouped_df['freq_y']

# drop the 'freq_x' and 'freq_y' columns
grouped_df = grouped_df.drop(['freq_x', 'freq_y'], axis=1)

# sort the dataframe by 'freq' in descending order
grouped_df = grouped_df.sort_values(by='freq', ascending=False)

# reset the index of the dataframe
grouped_df = grouped_df.reset_index(drop=True)
grouped_df

Loading in the analogy questions available on: http://corpus.nytud.hu/efnilex-vect/data/questions-words-hu.txt

In [1]:
# Vocabulary set of the analogy questions
with open('analogy.txt', 'r', encoding="utf-8") as f:
    # read in the lines and split them into words
    words = set()
    for line in f:
        try:
            w1, w2, w3, w4 = line.strip().split()
            words.update([w1, w2, w3, w4])
        except:
            pass

# Vocabulary set of the static embedding from efnilex (http://corpus.nytud.hu/efnilex-vect/)
# Creation of 'keys.txt'
with open('keys.txt', 'r', encoding="utf-8") as f:
    # read in the lines and split them into words
    efnilex = set()
    for line in f:
        word = line.strip()
        efnilex.add(word)

Restricting the vocabulary to words that appear at least 100 times according to the frequency table

In [ ]:
filtered = grouped_df[grouped_df.freq > 100]
efni = filtered[filtered['word'].isin(efnilex)]
efni = efni.sort_values('freq', ascending=False).reset_index()
efni = efni.drop(columns=['index'])

restricted = set(efni['word'])
with open('restricted_vocab.txt', 'w',encoding="utf-8") as f:
    for item in restricted:
        f.write('%s\n' % item)

# Extracting word vectors from huBERT

Extracting word vectors from the first layer, using mean pooling (with attention mask)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")

# Importing the vocabulary to a list
my_file = open('restricted_vocab.txt', 'r',encoding="utf-8")
data = my_file.read()
keys = data.replace('\n', ' ').split(" ")
my_file.close()

d = dict()
# The batch size 128 proved to be the most efficient
for i in tqdm(range(0,3800)):
    words=keys[(i)*128:(i+1)*128]
    encoded = tokenizer(words, padding=True, return_tensors="pt")
    bert_output = model(**encoded, output_hidden_states=True)
    embedding_output = bert_output['hidden_states'][0]
    # Mask out the padding tokens
    attention_mask = encoded['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(embedding_output.size()).float()
    masked_output = embedding_output * mask

    # Compute the mean of the non-padding tokens along the sequence axis
    pooled_output = torch.sum(masked_output, 1) / torch.clamp(torch.sum(mask, 1), min=1e-9)
    for i, word in enumerate(words):
        # Get the embedding for the ith word
        embedding = pooled_output[i]
        # Convert the embedding to a numpy array
        embedding = np.asarray(embedding.detach())
        # Add the embedding to the dictionary
        d[word] = embedding

Extracting word vectors from all layers, mean pooling them separately, then mean pooling the resulted tensor

In [ ]:
import numpy as np

# Define a dictionary to store the embeddings
d = dict()

# Loop over the words in batches
for i in tqdm(range(3000,4000)):
    words = keys[(i)*128:(i+1)*128]

    # Encode the words using the tokenizer
    encoded = tokenizer(words, padding=True, return_tensors="pt")

    # Get the outputs of all the layers of the model
    hubert_output = model(**encoded, output_hidden_states=True)
    hidden_states = hubert_output['hidden_states']

    # Loop over the layers and apply attention-based pooling
    layer_pooled_outputs = []
    for layer_output in hidden_states:
        # Compute the attention scores
        attention_scores = torch.matmul(layer_output, layer_output.transpose(-1, -2))

        # Mask out the padding tokens
        attention_mask = encoded['attention_mask']
        mask = attention_mask.unsqueeze(1) * attention_mask.unsqueeze(2)
        attention_scores = attention_scores.masked_fill(mask == 0, -1e9)

        # Normalize the attention scores
        attention_probs = torch.softmax(attention_scores, dim=-1)

        # Apply attention-based pooling
        layer_pooled_output = torch.matmul(attention_probs, layer_output)

        # Mask out the padding tokens
        attention_mask = encoded['attention_mask']
        mask = attention_mask.unsqueeze(-1).expand(layer_pooled_output.size()).float()
        layer_pooled_output = layer_pooled_output * mask

        # Compute the mean of the non-padding tokens along the sequence axis
        layer_pooled_output = torch.sum(layer_pooled_output, 1) / torch.clamp(torch.sum(mask, 1), min=1e-9)

        layer_pooled_outputs.append(layer_pooled_output)

    # Stack the layer pooled outputs into a tensor
    stacked_layer_pooled_outputs = torch.stack(layer_pooled_outputs, dim=1)

    # Compute mean pooling across the tensor's second dimension (the layer dimension)
    pooled_output = torch.mean(stacked_layer_pooled_outputs, dim=1)

    # Create a dictionary to store the embeddings
    for i, word in enumerate(words):
        # Get the embedding for the ith word
        embedding = pooled_output[i]
        # Convert the embedding to a numpy array
        embedding = np.asarray(embedding.detach())
        # Add the embedding to the dictionary
        d[word] = embedding

Storing a dictionary as a word2vec compatible binary file

In [ ]:
def save_word2vec_format(fname, vocab, vector_size, binary=True):
    total_vec = len(vocab)
    with gensim.utils.open(fname, 'wb') as fout:
        print(total_vec, vector_size)
        fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
        # store in sorted order: most frequent words at the top 
        for word, row in tqdm(vocab.items()):
            if binary:
                row = row.astype(np.float32)
                fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
            else:
                fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))

In [ ]:
save_word2vec_format(binary=True, fname='test_99.bin', vocab=d, vector_size=768)

# PCA

With 'd' dictionary containing n (in our case 768) dimensional word embeddings, the next code creates a d1 dictionary that contains embeddings for the same words but with 300 (can be easily changed) dimensions

In [ ]:
import numpy as np
from numpy import linalg as la
import pickle
    
l=list(d.values())
ma=np.array(l)

np.random.seed(42)


def flip_signs(A, B):
    """
    utility function for resolving the sign ambiguity in SVD
    http://stats.stackexchange.com/q/34396/115202
    """
    signs = np.sign(A) * np.sign(B)
    return A, B * signs


# Let the data matrix X be of n x p size,
# where n is the number of samples and p is the number of variables
X = ma
# Let us assume that it is centered
X -= np.mean(X, axis=0)

# the p x p covariance matrix
C = np.cov(X, rowvar=False)
print("C = \n", C)
# C is a symmetric matrix and so it can be diagonalized:
l, principal_axes = la.eig(C)
# sort results wrt. eigenvalues
idx = l.argsort()[::-1]
l, principal_axes = l[idx], principal_axes[:, idx]
# the eigenvalues in decreasing order
print("l = \n", l)
# a matrix of eigenvectors (each column is an eigenvector)
print("V = \n", principal_axes)
# projections of X on the principal axes are called principal components
principal_components = X.dot(principal_axes)
print("Y = \n", principal_components)

# we now perform singular value decomposition of X
# "economy size" (or "thin") SVD
U, s, Vt = la.svd(X, full_matrices=False)
V = Vt.T
S = np.diag(s)
 
# PCA to reduce dimensionality to k
d1 = dict()
k=300
PC_k = principal_components[:, 0:k]
US_k = U[:, 0:k].dot(S[0:k, 0:k])
j=0
for i in d.keys():
    d1[i]=US_k[j]
    j+=1

Extracting sentences from Hungarian Wikipedia that contains the selected ambigous words
The Hungarian Wikipedia is available here:

In [ ]:
# Self-made set of Hungarian ambigous words
with open('../hun_ambigous.txt',encoding='UTF-8') as file:
    words = [line.rstrip() for line in file]

# The Hungarian Wikipedia separated in lines
with open('../hun_wiki/hun_wiki1.txt',encoding='UTF-8') as file:
    lines1 = [line.rstrip() for line in file]
with open('../hun_wiki/hun_wiki2.txt',encoding='UTF-8') as file:
    lines2 = [line.rstrip() for line in file]
with open('../hun_wiki/hun_wiki3.txt',encoding='UTF-8') as file:
    lines3 = [line.rstrip() for line in file]
with open('../hun_wiki/hun_wiki4.txt',encoding='UTF-8') as file:
    lines4 = [line.rstrip() for line in file]
with open('../hun_wiki/hun_wiki5.txt',encoding='UTF-8') as file:
    lines5 = [line.rstrip() for line in file]
lines=lines1+lines2+lines3+lines4+lines5

# Listing all the appearances of the ambigous words in Hungarian Wikipedia
separated=[ [] for _ in range(51) ]
for i,word in enumerate(words):
    for j in lines:
        if ' '+word+' ' in j:
            separated[i].append(j)
    print(i)

Selecting example sentences for both sense

In [ ]:
# The next code were used to print out 100 random appearance of an ambigous word 
import random

random.sample(separated[0],100)

# From here, I selected 15-15 sentences for both senses in the following form
toll1 = ['A konyhában ezalatt a lányok megpróbálkoznak az átokhoz szükséges bájital összeállításával, azonban néhány gyógynövény és egy foltos bagolytól származó toll hiányzik az otthoni készletből.',
'Tehát ha a mérleg serpenyője, a szív és a toll egyensúlyban áll, akkor a lélek bebocsáttatást nyert Ozirisz halhatatlan birodalmába.',
 'De ha vákuumban végezzük a kísérletet, akkor a toll és a kalapács ugyanolyan gyorsan esik a talaj felé.',
 'Hiányzik a koponya nagyobb része, lábai hosszúak és erősek, tollazata nem látható, csak tüzetes vizsgálatok után lehetett kijelenteni, hogy az alkarcsonthoz néhány toll is csatlakozik.',
 'A homlokán barnás-zöld csík található a viaszhártyáján pedig pár darab piros toll is lehet, de ez csak az idősebb egyedekre jellemző.',
  'Szárnya 39-45 centiméter, farka 27-30 centiméter, a két középső toll kivételével mindegyik faroktollon feltűnően széles vörös sáv húzódik.',
  'A tollszár csomókat a toll tüszőkhöz kapcsolódó ínszalagok hozták létre, és mivel tüszőkből nem alakultak ki pikkelyek, a szerzők kizárták annak lehetőségét, hogy a karon hosszú, jelzésre szolgáló pikkelyek helyezkedtek el.',
 'Egy jó állapotban megőrződött részleges csontváz alapján ismert, melynek farkához négy hosszú, szárból és zászlókból álló toll lenyomata kapcsolódik.',
 'A nekem tollazata hasonló, de a hím csőrre valamivel hosszabb mint a tojó, és fején sötétebb toll található.',
 'A főzet már majdnem készen van a konyhában, s már csak a toll hiányzik az összetevők közül.',
 'A jelentős nemi kétalakúság jellemzi a Sphecotheres-fajokat: a hímek begyi és hasi részei olajzöldek, a fejük fekete, pofájukon pedig a toll nélküli bőr élénk vörös; a tojók kevésbé feltűnők, felül barnásak, alul pedig fehérek sötét csíkozással, a toll nélküli bőrük és csőrük szürkésfekete.',
 'A fiatal hímek fejecskéjén az első néhány vörös toll már a 3-4 hónapos korban megjelenik.',
 'A nászidőszakban színes tollak nőnek a hímek fején, toll nélküli pofáján a bőre narancssárgává válik, begye megfeketedik, és nyaka körül a nyakfodorhoz hasonló gallér alakul ki.',
 'Rövid csőre csonkának hat, és tövét toll borítja.',
 'A szárnyak és a farok sötét barna, a legszélső farok toll fehér.'
]
toll2 = ['A toll olyan íróeszköz, amely a tintának az írófelületre (leggyakrabban papírra) való felhordására alkalmas írás vagy rajzolás útján.',
 'A jegyzettömb papírból, vagy lemosható műanyagból készül, míg a toll egy speciális gömb formájú kupakkal van ellátva, hogy megakadályozza a bírót a sérüléstől.',
 'Csupán a vonal hordozója: a tus, ceruza vagy toll nyoma és a papír a materiális elem."',
  'Egy toll alakú eszköz segíti a felhasználót a készülék használatában.',
 'Mivel többnyire jobbról balra írják, praktikusabb bal kézzel írni, ellenkező esetben ugyanis a toll hegye átlyukasztja a papírt, az író kéz pedig elmaszatolja a tintát.',
 'Amikor a toll hegye érintkezett a táblával, akkor a mátrixban az adott ponton elhelyezkedő mikrofon érzékelte a hangot, és továbbította az adatot a számítógép felé.',
 'Habár a személyi számítógép és a billentyűzet használata egyre inkább teret nyert mindennapi életünkben, a toll még mindig a hétköznapok legfontosabb íróeszköze maradt.',
 'Hamarosan felismerte, hogy a golyó formájának precizitása nagyban befolyásolja a toll minőségét, ezért szigorú ellenőrzést vezetett be.',
 'A kereskedelmi forgalomba kerülő készülékek közül az 1989-ben piacra dobott Grid Systems GRiDPAD-ja volt az, ami legjobban egyesítette mindazt formában, amit alapvetően a táblagépekben ma is meghatározónak tartunk, de a 10 hüvelykes kijelző itt még monokróm, és beépített toll segítségével volt lehetőség az adatbevitelre.',
 'Az űrtollat 1967-től használhatták az amerikai űrhajósok, 1969-ben már a szovjet űrügynökség is érdeklődött a toll iránt.',
  'A toll márkanevében utalást tett családnevére is, illetve angolul a gördülő megoldású tollra is, így lett az kontinentális Európa első saját golyóstollának neve „GO”PEN.',
  '1938-ban Bíró László, magyar újságíró feltalált egy olyan íróeszközt, amelyben a tollhegy végén egy apró golyó forgott szabadon és vitte fel a toll belsejében tárolt tintát a papírra.',
 'Találmányát folyamatosan tökéletesítve 1906-ban London északi részén Tottenhamban létrehozta a stencilgépek, festékek, hengerek és a kerekes toll gyártására szakosodott üzemét.',
  'Mégis, a nagy technikai fejlődés ellenére – ami a toll és papír használatától a számítógép-hálózatokhoz vezetett – az egyes cégek könyvelésének lényeges meghatározója, a könyvelés költsége mégsem csökkent jelentősen.',
 'A gép első konstrukciója főleg azért volt nagyon különleges, mert az még akkor készült el, amikor még nem volt olyan léptetőmotor, amit a toll mozgatásához alkalmazni lehetett volna.'
]

Extracting vectors

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
db=0
sent = ['A jelentős nemi kétalakúság jellemzi a Sphecotheres-fajokat: a hímek begyi és hasi részei olajzöldek, a fejük fekete, pofájukon pedig a toll nélküli bőr élénk vörös; a tojók kevésbé feltűnők, felül barnásak, alul pedig fehérek sötét csíkozással, a toll nélküli bőrük és csőrük szürkésfekete.',
 'A fiatal hímek fejecskéjén az első néhány vörös toll már a 3-4 hónapos korban megjelenik.',
 'A nászidőszakban színes tollak nőnek a hímek fején, toll nélküli pofáján a bőre narancssárgává válik, begye megfeketedik, és nyaka körül a nyakfodorhoz hasonló gallér alakul ki.',
 'Rövid csőre csonkának hat, és tövét toll borítja.',
 'A szárnyak és a farok sötét barna, a legszélső farok toll fehér.'
    ]
for sentence in sent:
    # List of example sentences containing the ambiguous word
    ambiguous_word = "toll"  # The ambiguous word to extract vector for
    # Initialize list to store encoded sentences
    encoded_sentences = []

    indices = []

    # Tokenize the sentence and find the position of the ambiguous word
    tokens = tokenizer.tokenize(sentence)
    word_indices = [i for i, token in enumerate(tokens) if token == ambiguous_word]

    # Encode the sentence and mark the word position with special tokens [CLS] and [SEP]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    word_indices = [idx + 1 for idx in word_indices]  # Shift word indices by 1 to account for [CLS] token
    indices.append(word_indices[:1])
    encoded_sentences.append(torch.tensor(token_ids))

    # Perform mean pooling on all layers
    all_layers = []
    for encoded_sentence, word_indice in zip(encoded_sentences, indices):
        with torch.no_grad():
            outputs = model(encoded_sentence.unsqueeze(0), output_hidden_states=True)
            hidden_states = outputs['hidden_states']
            pooled_output = torch.mean(torch.stack(hidden_states), dim=0)
            all_layers.append(pooled_output.squeeze()[word_indice])

    # Perform mean pooling on the resulting vectors
    word_vector = torch.mean(torch.stack(all_layers), dim=0)

    # Convert the word vector to a numpy array
    word_vector = word_vector.numpy()

Evaluating (First with Euclidian distance, then with cosine similarity)

In [ ]:
import numpy as np
w='toll'

# Given arrays
array1 = d_sep[f'{w}1']
array2 = d_sep[f'{w}2']
array3 = d[f'{w}']

# Construct the coefficient matrix
A = np.vstack([array1, array2]).T

# Solve the least squares problem
coefficients, residuals, _, _ = np.linalg.lstsq(A, array3, rcond=None)

# Calculate the linear combination
linear_combination = np.dot(A, coefficients)

# Calculate the distance between linear_combination and array3
distance = np.linalg.norm(linear_combination - array3)

print("Coefficients:", coefficients)
print("Distance:", distance)

In [ ]:
c1 = np.linspace(0.0,10,num=1000)
c2 = np.linspace(0.0,10,num=1000)

w = 'rák'
# Assuming you have three numpy arrays: arr1, arr2, arr3
arr1 = d_sep[f'{w}1']
arr2 = d_sep[f'{w}2']
arr3 = d[f'{w}']
maximum = 0
a=0
b=0
for i in c1:
    for j in c2:
        weighted_sum = i * arr1 + j * arr2
        similarity = cosine_similarity(weighted_sum.reshape(1, -1), arr3.reshape(1, -1))
        if similarity > maximum:
            maximum = similarity
            a = i
            b = j
print(similarity, a, b)